In [ ]:
# This section has to be invoked once per MLRun/Iguazio upgrade
from mlrun.runtimes import RemoteSparkRuntime
RemoteSparkRuntime.deploy_default_image()

In [1]:
from mlrun import code_to_function

sj = code_to_function("read_snowflake", 
                     filename="read_snowflake.py", 
                     handler="read_snowflake",
                     kind="spark"
                     )
sj.with_driver_requests(mem="4096m")
sj.with_executor_requests(mem="4096m")
sj.spec.use_default_image = True
sj.with_igz_spark()

In [2]:
sj.export()

> 2022-03-10 12:23:08,129 [info] function spec saved to path: function.yaml


In [ ]:
from mlrun import import_function
fx = import_function("hub://read_snowflake")

fx.with_igz_spark()

# Create secrets for Snowflake URL, User, and Password via API or UI 
fx.run(params={"url": "sfURL",
               "user": "sfUser",
               "password": "sfPassword",
               "db": "snowflake_sample_data", 
               "schema": "tpch_sf1", 
               "warehouse": "compute_wh", 
               "query": "select * from customer limit 100", 
               "target": "v3io://bigdata/features/customer.parquet"}).with_secrets('kubernetes', ['sfURL', 'sfUser', 'sfPassword'])